# Data Preparation and Cleaning

In this notebook, I clean the datasets and combine them into a single csv file that can be used later for feature generation.

In [1]:
# Import packages
import sys
sys.path.append('../../College_Basketball')

import pandas as pd
import collegebasketball as cbb
cbb.__version__

'0.3'

## Load in the Game Scores Data

First, we will load in the games scores data from csv files we created earlier. Later, we'll join this data to the team stats datasets. 

In [2]:
# Location of the data
scores_path = '../Data/Scores/'

# Initialize some variables
regular_season = {}
march_madness = {}
this_year = 2019

# We need to first join datasets from the same year
for year in range(2002, this_year):
    
    # Load the scores datasets
    regular_season[year] = pd.read_csv(scores_path + str(year) + '_regular_season.csv')
    if year < this_year - 1:
        march_madness[year] = pd.read_csv(scores_path + str(year) + '_march.csv')

## Cleaning the Data

Next, we need to edit the school names in the kenpom, basic stats and T-Rank datasets to ensure that they match up with the school names from the scores dataset. We will verify that the names match using the `cbb.check_for_missing_names` command. It checks that each school name in the given team statistics dataset (kenpom, basic stats or T-Rank) is present in the game scores dataset.

In [3]:
# The location where the files will be saved
path = '../Data/'

# Store a dataframe of kenpom data for each year in a list
kenpom_data = {}
TRank_data = {}
stats_data = {}

# We need to clean each statistics data set
for year in range(2002, this_year):
    
    # Load this year's data and clean up the school names to match up with scores data
    data_kenpom = pd.read_csv('{0}Kenpom/{1}_kenpom.csv'.format(path, year))
    kenpom_data[year] = cbb.update_kenpom(data_kenpom)
    assert len(cbb.check_for_missing_names(regular_season[year], kenpom_data[year], False)) == 0
    
    # TRank data starts in 2008 and the team name column is called school instead of team
    if year > 2007:
        data_TRank =  pd.read_csv('{0}TRank/{1}_TRank.csv'.format(path, year))
        TRank_data[year] = cbb.update_TRank(data_TRank)
        assert len(cbb.check_for_missing_names(regular_season[year], TRank_data[year], False)) == 0
        
    # Basic stats data starts in 2010
    if year > 2009:
        data_stats =  pd.read_csv('{0}SportsReference/{1}_stats.csv'.format(path, year))
        data_stats = data_stats.rename(index=str, columns={'School': 'Team'})
        stats_data[year] = cbb.update_basic(data_stats)
        assert len(cbb.check_for_missing_names(regular_season[year], stats_data[year], False)) == 0

In [4]:
# Lets take a quick look at one of the datasets
kenpom_data[2013].head()

,Rank,Team,Seed,Conf,Wins,Losses,AdjEM,AdjO,AdjO Rank,AdjD,...,Luck,Luck Rank,OppAdjEM,OppAdjEM Rank,OppO,OppO Rank,OppD,OppD Rank,NCSOS AdjEM,NCSOS AdjEM Rank
0,1,Louisville,1.0,BE,35,5,32.92,117.7,7,84.8,...,-0.016,230,10.23,7,107.4,11,97.2,6,2.15,100
1,2,Florida,3.0,SEC,29,8,31.18,117.2,9,86.0,...,-0.089,332,7.00,41,105.6,63,98.6,36,4.09,62
2,3,Indiana,1.0,B10,29,7,29.31,120.8,2,91.5,...,-0.021,239,8.86,17,106.7,23,97.8,14,-5.16,294
3,4,Michigan,4.0,B10,31,8,27.86,121.9,1,94.0,...,-0.007,197,11.13,3,108.2,5,97.0,3,-3.09,257
4,5,Gonzaga,1.0,WCC,32,3,27.81,120.5,3,92.7,...,0.026,116,4.37,87,105.4,75,101.0,102,5.03,39


## Joining the Datasets

Now that the school names from each data set matches up, we can join the kenpom and score data to form a single csv file. Additionally, the march scores datasets contained some games that were from postseason tournament games other than the NCAA Tournamnet (for example NIT games are currently in the march scores data). Since, we joined the kenpom data (containing the team seeds) to the game scores, we can find and remove non-NCAA Tounament games from the march datasets by removing games that are not between teams with an NCAA Tournament seed. We will remove these games from the march datasets and add them in the regular season data.

In [5]:
# Save the paths to the data 
save_path = '../Data/Combined_Data/'

# Save the joined tables in dictionaries
regular = {}
march = {}

# We need to first join datasets from the same year
for year in range(2002, this_year):
    
    # Join the dataframes to get kenpom for both home and away team
    regular[year] = pd.merge(regular_season[year], kenpom_data[year], left_on='Home', right_on='Team', sort=False)
    regular[year] = pd.merge(regular[year], kenpom_data[year], left_on='Away', right_on='Team', 
                             suffixes=('_Home', '_Away'), sort=False)
    
    # Do the same for the march data (No march data for this year yet)
    if year < this_year - 1:
        march[year] = pd.merge(march_madness[year], kenpom_data[year], left_on='Home', right_on='Team', sort=False)
        march[year] = pd.merge(march[year], kenpom_data[year], left_on='Away', right_on='Team', 
                                 suffixes=('_Home', '_Away'), sort=False)

        # Move non-tournament games to regular season data
        other_games = march[year][march[year]['Seed_Home'].isnull()]
        regular[year] = pd.concat([regular[year], other_games], ignore_index=True)
        march[year].drop(other_games.index, inplace=True)
    
    # Add a column to indicate the year
    regular[year].insert(0, 'Year', year)
    if year < this_year - 1:
        march[year].insert(0, 'Year', year)
        
# Combine the data for every year
regular_df = pd.concat(regular, ignore_index=True)
march_df = pd.concat(march, ignore_index=True)

# Save the data to csv files
regular_df.to_csv('{0}Kenpom.csv'.format(save_path))
    
# Lets take a look at the data set
print("There are {} games in the Kenpom dataset.".format(len(regular_df)))
regular_df.head()

There are 87521 games in the Kenpom dataset.


,Year,Home,Away,Home_Score,Away_Score,Rank_Home,Team_Home,Seed_Home,Conf_Home,Wins_Home,...,Luck_Away,Luck Rank_Away,OppAdjEM_Away,OppAdjEM Rank_Away,OppO_Away,OppO Rank_Away,OppD_Away,OppD Rank_Away,NCSOS AdjEM_Away,NCSOS AdjEM Rank_Away
0,2002,Maryland,Arizona,67,71,3,Maryland,1.0,ACC,32,...,0.079,15,14.22,1,111.3,1,97.1,3,17.56,1
1,2002,Florida,Arizona,71,75,7,Florida,5.0,SEC,22,...,0.079,15,14.22,1,111.3,1,97.1,3,17.56,1
2,2002,Wyoming,Arizona,60,68,67,Wyoming,11.0,MWC,22,...,0.079,15,14.22,1,111.3,1,97.1,3,17.56,1
3,2002,USC,Arizona,80,97,12,USC,4.0,P10,22,...,0.079,15,14.22,1,111.3,1,97.1,3,17.56,1
4,2002,USC,Arizona,71,81,12,USC,4.0,P10,22,...,0.079,15,14.22,1,111.3,1,97.1,3,17.56,1


Now we will clean up the team names in the T-Rank data and join it with the game scores data. Additionally, we need to join these data sets with the team Kenpom statistics. This join is necessary because we need to use the Tournament seed attribute in order to clean up the march dataset to only include NCAA Tournament games. It will also be beneficial down the road, during feature generation, for us to have the Kenpom AdjEM stat for each team as a way to judge what outcome of a game is considered an upset.

In [6]:
# Save the paths to the scores data 
save_path = '../Data/Combined_Data/'

# Save the joined tables in dictionaries
regular = {}
march = {}

# We need to first join datasets from the same year
for year in range(2008, this_year):
    
    # Get only the columns we need from the kenpom data
    kp = kenpom_data[year][['Team', 'AdjEM', 'Seed']]
    
    # Join the dataframes to get TRank data and kenpom (seed, adj_em) for both home and away team
    regular[year] = pd.merge(regular_season[year], TRank_data[year], left_on='Home', right_on='Team', sort=False)
    regular[year] = pd.merge(regular[year], TRank_data[year], left_on='Away', right_on='Team', 
                             suffixes=('_Home', '_Away'), sort=False)
    regular[year] = pd.merge(regular[year], kp, left_on='Home', right_on='Team', sort=False)
    regular[year] = pd.merge(regular[year], kp, left_on='Away', right_on='Team', 
                             suffixes=('_Home', '_Away'), sort=False)
    
    # Do the same for the march data (No march data for this year yet)
    if year < this_year - 1:
        march[year] = pd.merge(march_madness[year], TRank_data[year], left_on='Home', right_on='Team', sort=False)
        march[year] = pd.merge(march[year], TRank_data[year], left_on='Away', right_on='Team', 
                                 suffixes=('_Home', '_Away'), sort=False)
        march[year] = pd.merge(march[year], kp, left_on='Home', right_on='Team', sort=False)
        march[year] = pd.merge(march[year], kp, left_on='Away', right_on='Team', 
                                 suffixes=('_Home', '_Away'), sort=False)

        # Move non-tournament games to regular season data
        other_games = march[year][march[year]['Seed_Home'].isnull()]
        regular[year] = pd.concat([regular[year], other_games], ignore_index=True)
        march[year].drop(other_games.index, inplace=True)
    
    # Add a column to indicate the year
    regular[year].insert(0, 'Year', year)
    if year < this_year - 1:
        march[year].insert(0, 'Year', year)
    
# Combine the data for every year
regular_df = pd.concat(regular, ignore_index=True)
march_df = pd.concat(march, ignore_index=True)

# Save the data to csv files
regular_df.to_csv('{0}TRank.csv'.format(save_path))
    
# Lets take a look at one of the data sets
print("There are {} games in the T-Rank dataset.".format(len(regular_df)))
regular_df.head()

There are 58957 games in the T-Rank dataset.


,Year,Home,Away,Home_Score,Away_Score,Rk_Home,Team_Home,Conf_Home,G_Home,W_Home,...,Adj T._Away,Adj T. Rank_Away,WAB_Away,WAB Rank_Away,Team_Home,AdjEM_Home,Seed_Home,Team_Away,AdjEM_Away,Seed_Away
0,2008,UT-Martin,Memphis,71,102,252,UT-Martin,OVC,31,17,...,70.2,72,8.9,5,UT-Martin,-8.10,NaN,Memphis,31.51,1.0
1,2008,Richmond,Memphis,63,80,143,Richmond,A10,31,16,...,70.2,72,8.9,5,Richmond,1.48,NaN,Memphis,31.51,1.0
2,2008,Siena,Memphis,58,102,97,Siena,MAAC,34,23,...,70.2,72,8.9,5,Siena,7.99,13.0,Memphis,31.51,1.0
3,2008,Pepperdine,Memphis,53,90,241,Pepperdine,WCC,31,11,...,70.2,72,8.9,5,Pepperdine,-6.39,NaN,Memphis,31.51,1.0
4,2008,Alabama-Birmingham,Memphis,56,94,66,Alabama-Birmingham,CUSA,34,23,...,70.2,72,8.9,5,Alabama-Birmingham,12.07,NaN,Memphis,31.51,1.0


Lastly, we will run the same process for the basic statistics as we did for the T-Rank data.

In [7]:
# Save the paths to the scores data 
save_path = '../Data/Combined_Data/'

# Save the joined tables in dictionaries
regular = {}
march = {}

# We need to first join datasets from the same year
for year in range(2010, this_year):
    
    # Get only the columns we need from the kenpom data
    kp = kenpom_data[year][['Team', 'AdjEM', 'Seed']]
    
    # Join the dataframes to get basic statistics data and kenpom (seed, adj_em) for both home and away team
    regular[year] = pd.merge(regular_season[year], stats_data[year], left_on='Home', right_on='Team', sort=False)
    regular[year] = pd.merge(regular[year], stats_data[year], left_on='Away', right_on='Team', 
                             suffixes=('_Home', '_Away'), sort=False)
    regular[year] = pd.merge(regular[year], kp, left_on='Home', right_on='Team', sort=False)
    regular[year] = pd.merge(regular[year], kp, left_on='Away', right_on='Team', 
                             suffixes=('_Home', '_Away'), sort=False)
    
    # Do the same for the march data (No march data for this year yet)
    if year < this_year - 1:
        march[year] = pd.merge(march_madness[year], stats_data[year], left_on='Home', right_on='Team', sort=False)
        march[year] = pd.merge(march[year], stats_data[year], left_on='Away', right_on='Team', 
                                 suffixes=('_Home', '_Away'), sort=False)
        march[year] = pd.merge(march[year], kp, left_on='Home', right_on='Team', sort=False)
        march[year] = pd.merge(march[year], kp, left_on='Away', right_on='Team', 
                                 suffixes=('_Home', '_Away'), sort=False)

        # Move non-tournament games to regular season data
        other_games = march[year][march[year]['Seed_Home'].isnull()]
        regular[year] = pd.concat([regular[year], other_games], ignore_index=True)
        march[year].drop(other_games.index, inplace=True)
    
    # Add a column to indicate the year
    regular[year].insert(0, 'Year', year)
    if year < this_year - 1:
        march[year].insert(0, 'Year', year)
    
# Combine the data for every year
regular_df = pd.concat(regular, ignore_index=True)
march_df = pd.concat(march, ignore_index=True)

# Save the data to csv files
regular_df.to_csv('{0}Basic.csv'.format(save_path))
    
# Lets take a look at one of the data sets
print("There are {} games in the basic statistics dataset.".format(len(regular_df)))
regular_df.head()

There are 48179 games in the basic statistics dataset.


,Year,Home,Away,Home_Score,Away_Score,Team_Home,G_Home,SRS_Home,SOS_Home,Tm._Home,...,STL_Away,BLK_Away,TOV_Away,PF_Away,Team_Home,AdjEM_Home,Seed_Home,Team_Away,AdjEM_Away,Seed_Away
0,2010,Florida International,UNC,72,88,Florida International,32,-12.81,-2.74,2176,...,250,210,562,564,Florida International,-14.45,NaN,UNC,13.39,NaN
1,2010,Albany (NY),UNC,70,87,Albany (NY),32,-11.94,-5.53,2007,...,250,210,562,564,Albany (NY),-13.16,NaN,UNC,13.39,NaN
2,2010,William & Mary,UNC,72,80,William & Mary,33,2.82,1.63,2213,...,250,210,562,564,William & Mary,6.58,NaN,UNC,13.39,NaN
3,2010,Valparaiso,UNC,77,88,Valparaiso,32,-2.90,0.34,2344,...,250,210,562,564,Valparaiso,-0.92,NaN,UNC,13.39,NaN
4,2010,Wake Forest,UNC,82,69,Wake Forest,31,11.45,8.85,2257,...,250,210,562,564,Wake Forest,14.12,9.0,UNC,13.39,NaN
